Getting usefull data using eurostat package

In [16]:
import eurostat

# Dataset 	GERD by sector of performance and socioeconomic objectives rd_e_gerdsobj92
# sectperf: TOTAL All sectors
# sectperf: BES Business enterprise sector
# sectperf: GVT Government sector
# sectperf: HES Higher education sector
# sectperf: PNP Private non-profit sector

# unit = unit of measure
# EUR_HAB = Euro per inhabitant
# MIO_EUR = Million Euro
# MIO_PPS = Million Purchasing Power Standard
# MIO_PPS_KP05 = Million Purchasing Power Standard per capita, 2005 prices

# geo = country
# EU27_2020 = European Union (27 countries)
# EA20 = Euro area (20 countries)


RD_EXP_BY_SECOTR = eurostat.get_data_df("rd_e_gerdsobj92",True)
print(RD_EXP_BY_SECOTR.head())
print(RD_EXP_BY_SECOTR["geo\TIME_PERIOD"].unique())

countries = RD_EXP_BY_SECOTR["geo\TIME_PERIOD"].unique()

#Patent applications https://ec.europa.eu/eurostat/databrowser/view/pat_ep_ntot/default/table?lang=en&category=scitech.ipr.pat.pat_h
PAT_APPLICATIONS = eurostat.get_data_df("pat_ep_ntot",True)

  freq nabs92 sectperf     unit geo\TIME_PERIOD  1981_value 1981_flag  \
0    A  NBS01      BES  EUR_HAB              AT         0.0             
1    A  NBS01      BES  EUR_HAB              CH         NaN         :   
2    A  NBS01      BES  EUR_HAB              CZ         NaN         :   
3    A  NBS01      BES  EUR_HAB              EE         NaN         :   
4    A  NBS01      BES  EUR_HAB              ES         0.1             

   1982_value 1982_flag  1983_value  ... 2002_value  2002_flag 2003_value  \
0         NaN         :         NaN  ...        NaN          :        NaN   
1         NaN         :         NaN  ...        NaN          :        NaN   
2         NaN         :         NaN  ...        0.1                   0.1   
3         NaN         :         NaN  ...        0.0                   0.0   
4         0.1                   0.1  ...        NaN          :        1.0   

   2003_flag 2004_value  2004_flag 2005_value  2005_flag 2006_value  2006_flag  
0          :     